In [ ]:
import os
import glob

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (interactive but deprecated) or "%matplotlib inline" (static plots)
%matplotlib widget
from matplotlib import pyplot
import ipywidgets
import numpy as np

import pyfabm

In [ ]:
# Settings
fabm_yaml = "fabm/testcases/fabm-nersc-ecosmo.yaml"

xname = "temperature"
xvalues = np.linspace(0.0, 40.0, 100)
yname = "CO2/CO2_flux"

bottom_depth = 10
default_environment = {
    'temperature': 15,
    'practical_salinity': 35,
    'surface_downwelling_shortwave_flux': 50,
    'surface_downwelling_photosynthetic_radiative_flux': 25,
    'downwelling_photosynthetic_radiative_flux': 10,
    'density': 1020,
    'mole_fraction_of_carbon_dioxide_in_air': 414.2,
    'wind_speed': 2.0,
    'bottom_depth': bottom_depth,
    'depth': 0.5*bottom_depth,
    'pressure': 0.5*bottom_depth,
    'cell_thickness': bottom_depth
}

In [ ]:
model = pyfabm.Model(fabm_yaml)

model.cell_thickness = bottom_depth
for variable in model.dependencies:
    variable.value = default_environment.get(variable.name, 0.0)

# Initialize model (this also verifies that all dependencies have been fulfilled)
assert model.start(), 'Model failed to start: %s' % pyfabm.getError()
    

In [ ]:
yvar = model.diagnostic_variables.find(yname)
xvar = model.dependencies.find(xname)
yvalues = np.empty_like(xvalues)
for i, x in enumerate(xvalues):
    xvar.value = x
    model.getRates()
    yvalues[i] = yvar.value

In [ ]:
fig, ax = pyplot.subplots()
ax.plot(xvalues, yvalues, '-')
ax.grid()
ax.set_xlabel(f"{xvar.long_name} ({xvar.units})");
ax.set_ylabel(f"{yvar.long_name} ({yvar.units})");
fig.tight_layout()

In [ ]:
# reset environment variable that we were varying
xvar.value = default_environment.get(xname, 0.0)

xname = "ECO/alfaPs"
yname = "ECO/primprod"
xvalues = np.linspace(0.0, 0.1, 100)

xvar = model.parameters.find(xname)

yvalues = np.empty_like(xvalues)
for i, x in enumerate(xvalues):
    xvar.value = x

    # changes in parameter values require the model to be restarted
    model.start()
    yvar = model.diagnostic_variables.find(yname)

    model.getRates()
    yvalues[i] = yvar.value

In [ ]:
fig, ax = pyplot.subplots()
ax.plot(xvalues, yvalues, '-')
ax.grid()
ax.set_xlabel(f"{xvar.long_name} ({xvar.units})");
ax.set_ylabel(f"{yvar.long_name} ({yvar.units})");
fig.tight_layout()